## 데이터 > calib_set.npy

In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
import os
from roboflow import Roboflow

try:
    # Roboflow API 설정
    rf = Roboflow(api_key="write_apikey")
    project = rf.workspace("workspace_name").project("candoit")
    version = project.version(3)

    # YOLOv8 데이터셋 다운로드
    dataset = version.download("yolov8")
    print(f"Dataset downloaded to: {dataset.location}")
except Exception as e:
    print("An error occurred while downloading the dataset:")
    print(e)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to candoit-3 in yolov8:: 100%|██████████| 19629/19629 [00:02<00:00, 7403.86it/s]

Dataset downloaded to: /content/candoit-3


In [3]:
import os
import numpy as np
from PIL import Image
import cv2

def preprocess_image(image, target_size=(640, 640)):
    # 이미지 크기 조정
    resized_image = cv2.resize(image, target_size)
    # 정규화 (0-255 범위를 0-1로 변환)
    normalized_image = resized_image / 255.0
    return normalized_image

# 이미지가 있는 디렉토리 경로
image_dir = "./candoit-3/train/images"

# 캘리브레이션 데이터셋을 위한 이미지 목록
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# 캘리브레이션 데이터셋 생성
calib_dataset = []

for image_file in image_files[:100]:  # 처음 100개 이미지만 사용
    image_path = os.path.join(image_dir, image_file)
    image = cv2.imread(image_path)
    if image is not None:
        preprocessed_image = preprocess_image(image)
        calib_dataset.append(preprocessed_image)

# NumPy 배열로 변환
calib_dataset = np.array(calib_dataset)

# .npy 파일로 저장
np.save('calib_set.npy', calib_dataset)

print(f"Calibration dataset saved to calib_set.npy with shape: {calib_dataset.shape}")

Calibration dataset saved to calib_set.npy with shape: (100, 640, 640, 3)


## pt파일 업로드 > onnx 변환

In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.3/887.3 kB 17.4 MB/s eta 0:00:00


In [5]:
import os

print(os.listdir('/content'))


['.config', 'calib_set.npy', 'candoit-3', 'best.pt', 'sample_data']


In [6]:
from ultralytics import YOLO

model = YOLO("/content/best.pt")
onnx_path = model.export(
    format="onnx",
    opset=11,
    dynamic=False,
    simplify=True,
    imgsz=640,
    half=False
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.36 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (21.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 131.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 299.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import onnx

# ONNX 파일 로드
model = onnx.load("best.onnx")

# 모델의 모든 노드 이름 출력
for node in model.graph.node:
    print(node.name)


/model.0/conv/Conv
/model.0/act/Sigmoid
/model.0/act/Mul
/model.1/conv/Conv
/model.1/act/Sigmoid
/model.1/act/Mul
/model.2/cv1/conv/Conv
/model.2/cv1/act/Sigmoid
/model.2/cv1/act/Mul
/model.2/Split
/model.2/m.0/cv1/conv/Conv
/model.2/m.0/cv1/act/Sigmoid
/model.2/m.0/cv1/act/Mul
/model.2/m.0/cv2/conv/Conv
/model.2/m.0/cv2/act/Sigmoid
/model.2/m.0/cv2/act/Mul
/model.2/m.0/Add
/model.2/Concat
/model.2/cv2/conv/Conv
/model.2/cv2/act/Sigmoid
/model.2/cv2/act/Mul
/model.3/conv/Conv
/model.3/act/Sigmoid
/model.3/act/Mul
/model.4/cv1/conv/Conv
/model.4/cv1/act/Sigmoid
/model.4/cv1/act/Mul
/model.4/Split
/model.4/m.0/cv1/conv/Conv
/model.4/m.0/cv1/act/Sigmoid
/model.4/m.0/cv1/act/Mul
/model.4/m.0/cv2/conv/Conv
/model.4/m.0/cv2/act/Sigmoid
/model.4/m.0/cv2/act/Mul
/model.4/m.0/Add
/model.4/m.1/cv1/conv/Conv
/model.4/m.1/cv1/act/Sigmoid
/model.4/m.1/cv1/act/Mul
/model.4/m.1/cv2/conv/Conv
/model.4/m.1/cv2/act/Sigmoid
/model.4/m.1/cv2/act/Mul
/model.4/m.1/Add
/model.4/Concat
/model.4/cv2/conv/Conv


## achor

In [8]:
import os
import numpy as np
from sklearn.cluster import KMeans

# 라벨 파일 경로 설정
label_dir = "candoit-3/train/labels"
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

# 바운딩 박스 정보 수집
bounding_boxes = []

for label_file in label_files:
    with open(os.path.join(label_dir, label_file), 'r') as file:
        for line in file:
            _, x_center, y_center, width, height = map(float, line.split())
            # 너비와 높이를 리스트에 추가 (이미지 크기에 상관없이 비율로 사용)
            bounding_boxes.append([width, height])

# NumPy 배열로 변환
bounding_boxes = np.array(bounding_boxes)

# K-means 클러스터링을 사용하여 anchor box 계산
num_anchors = 9  # 원하는 anchor box의 개수 (보통 9개 사용)
kmeans = KMeans(n_clusters=num_anchors, random_state=0)
kmeans.fit(bounding_boxes)

# 클러스터 중심이 anchor box 크기
anchors = kmeans.cluster_centers_
print("Calculated Anchor Boxes (width, height):")
print(anchors)


Calculated Anchor Boxes (width, height):
[[    0.34434     0.33015]
 [    0.43173     0.41563]
 [    0.29384     0.28205]
 [    0.62447     0.59113]
 [    0.23515     0.22402]
 [    0.38767      0.3716]
 [    0.47882     0.46508]
 [    0.54266     0.52476]
 [    0.54753     0.45088]]


정규화되지 않길 원하면 *640을 하자!

absolute_width = 640 * 0.381 ≈ 244
absolute_height = 640 * 0.365 ≈ 234


## 업로드 (npy파일)